# ResNet Example

Model definition and training code is adapted from https://github.com/kuangliu/pytorch-cifar.

In [ ]:
!(pip show torch-summary >& /dev/null || pip install --quiet torch-summary)

In [ ]:
!(test -d .git || test -d mp1 || git clone https://github.com/mike10004/csgy6953-mp1.git mp1)

In [ ]:
# change to correct branch here, if not main
!(test -d mp1 && cd mp1 && git switch flexy-resnet)
!(test -d mp1 && cd mp1 && git pull && git rev-parse --short HEAD)

In [ ]:
!(test -d mp1 && pip install --quiet --editable mp1)
import site
site.main()

In [ ]:
import dlmp1
import importlib
importlib.reload(dlmp1)
from pathlib import Path
print("checked importable:", dlmp1, "at", Path(dlmp1.__file__).parent)

In [ ]:
import os
import shutil
from typing import Optional

# set empty to disable saving
# note that the first path component MyDrive is required
GDRIVE_SAVE_DIR = "MyDrive/CS-GY 6953 DL/deep learning midterm project/checkpoints"

def prepare_mount() -> Optional[str]:
    save_path_root = "/content/gdrive"
    local_save_root = str(os.path.join(save_path_root, GDRIVE_SAVE_DIR)) 
    if GDRIVE_SAVE_DIR:
        try:
            from google.colab import drive
            drive.mount(save_path_root)
            return local_save_root
        except ImportError:
            print("(not saving because not in colab environment)")

LOCAL_GDRIVE_SAVE_PATH = prepare_mount()

def upload_checkpoint(checkpoint_file: Path, infix: str) -> Optional[str]:
    if LOCAL_GDRIVE_SAVE_PATH:
        filename = f"{checkpoint_file.stem}-{infix}{checkpoint_file.suffix}"
        dst_file = os.path.join(LOCAL_GDRIVE_SAVE_PATH, filename)
        shutil.copyfile(checkpoint_file, dst_file)
        return dst_file


In [ ]:
from dlmp1.models.resnet import CustomResNet
from dlmp1.models.resnet import BlockSpec
# noinspection PyPackageRequirements
import torchsummary

def create_model():
    return CustomResNet([
        BlockSpec(2, 64, stride=1),
        BlockSpec(5, 128, stride=2),
        BlockSpec(3, 256, stride=2),
    ]) 

def summarize_model():
    model = create_model()
    stats = torchsummary.summary(model, verbose=0)
    print(type(model).__name__, f"{stats.trainable_params/1_000_000:.1f}m trainable parameters ({stats.trainable_params})")
    del model

summarize_model()

In [ ]:
from dlmp1.main import Dataset
BATCH_SIZE_TRAIN = 128
DATASET = Dataset.acquire(BATCH_SIZE_TRAIN)


In [ ]:
import dlmp1.main
from dlmp1.main import TrainConfig

DO_TRAIN = False
CONFIG = TrainConfig(
    epoch_count=160,
    learning_rate=0.1,
    seed=987654321
)

TRAIN_RESULT = None
if DO_TRAIN:
    TRAIN_RESULT = dlmp1.main.perform(
        model_provider=create_model,
        dataset=DATASET,
        config=CONFIG,
    )

if TRAIN_RESULT is not None:
    CHECKPOINT_DST_PATH = upload_checkpoint(TRAIN_RESULT.checkpoint_file, TRAIN_RESULT.timestamp)
    if CHECKPOINT_DST_PATH:
        print(f"copied checkpoint file to", CHECKPOINT_DST_PATH)